### Set if you work in pycharm or in native jupyter environment (recommended): used for plotting

In [1]:
IN_PYCHARM : bool = False

### Preliminary imports and settings

In [2]:
# Imports
from math import isclose
from copy import deepcopy
from IPython.display import clear_output
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt

import contracts
import geometry as geo

import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html, ipython_draw_svg
from duckietown_world.world_duckietown.tile import get_lane_poses
from duckietown_world.world_duckietown.tile_utils import get_tile_at_point
from duckietown_world.rules.rule import evaluate_rules

from world.map_loading import load_driving_game_map
from world.skeleton_graph import get_skeleton_graph
from world.utils import get_lane_from_node_sequence

# Some settings
contracts.disable_all()
dw.logger.setLevel(50)
d = "out/"

class Person(dw.PlacedObject):

    def __init__(self, radius, *args, **kwargs):
        self.radius = radius
        dw.PlacedObject.__init__(self, *args, **kwargs)

    def draw_svg(self, drawing, g):
        # drawing is done using the library svgwrite
        c = drawing.circle(center=(0, 0), r=self.radius, fill='red')
        g.add(c)
        # draws x,y axes
        dw.draw_axes(drawing, g)

    def extent_points(self):
        # set of points describing the boundary
        L = self.radius
        return [(-L, -L), (+L, +L)]

INFO:geometry:version: 2.0.5
INFO:commons:version: 6.1.5
INFO:typing:version: 6.1.8
DEBUG:duckietown_world:duckietown-world version 6.2.7 path /home/chrigi/Master-Thesis-Code/driving-games/venv/lib/python3.8/site-packages


ModuleNotFoundError: No module named 'duckie_games.map_loading'

### Define some useful functions

In [ ]:
def draw_object(pl_ob: dw.PlacedObject, outdir=None, area=None, in_pycharm: bool = IN_PYCHARM):
    if in_pycharm:
        ipython_draw_html(po=pl_ob, outdir=outdir, area=area)
        clear_output(wait=True)
        ipython_draw_svg(m=pl_ob, outdir=outdir)
    else:
        ipython_draw_html(po=pl_ob, outdir=outdir, area=area)

def draw_graph(G0, pos=None):
    pos = pos or nx.spring_layout(G0)
    plt.figure(figsize=(12, 12))
    nx.draw(G0,pos,labels={node:node for node in G0.nodes()})
    def edge_label(a, b):
        datas = G0.get_edge_data(a, b)
        s = '%d edge%s' % (len(datas), 's' if len(datas)>=2 else '')
        for k, v in datas.items():
            if v:
                if 'label' in v:
                    s += '\n %s' % v['label']
                else:
                    s += '\n %s' %v
        return s
    edge_labels = dict([ ((a,b), edge_label(a,b)) for a,b in G0.edges()])
    nx.draw_networkx_edge_labels(G0,pos,edge_labels=edge_labels,font_color='red')
    plt.axis('off')
    plt.show()

### Better Visualisation of output

In [ ]:
%%html
<style>
pre {line-height: 90%}
</style>

### Import and draw the map

In [ ]:
map_name = "4way-double-intersection-only"
dirname = d + map_name
duckie_map = load_driving_game_map(map_name)
draw_object(duckie_map, outdir=dirname)

### Get skeleton graph (contains the lane segments and the graph interconnecting them)

In [ ]:
skeleton_graph = get_skeleton_graph(duckie_map)

### Get the map of the joined lane segments and draw it (check box "map lane segments" to see it)

In [ ]:
joined_lane_segments = skeleton_graph.root2

dirname = d + "4wayjoinedlane"
draw_object(joined_lane_segments, outdir=dirname)

### Get the graph of the topology

In [ ]:
topology_graph = skeleton_graph.G

### Draw the graph

In [ ]:
pos = {}
for n in topology_graph:
    q = topology_graph.nodes[n]['point'].as_SE2()
    t, _ = geo.translation_angle_from_SE2(q)
    pos[n] = t
draw_graph(topology_graph, pos=pos)

### Extract the shortest path from a start node to an end node as list

In [ ]:
start = "P30"
end = "P9"

path = nx.shortest_path(topology_graph, start, end)

def get_lanes(path, graph):
    edges = zip(path[:-1], path[1:])
    lanes =[]
    for a, b in edges:
        lane = graph.get_edge_data(a, b)[0]['lane']
        lanes.append(lane)
    return lanes

lanes = get_lanes(path, topology_graph)
print(lanes)

### Extract the lane segments and draw them

In [ ]:
lane_segments_to_follow = [joined_lane_segments.children[lane_name] for lane_name in lanes]

lane_points = []
po = dw.PlacedObject()
for lane_name, lane in zip(lanes, lane_segments_to_follow):
    po.set_object(lane_name, lane, ground_truth=dw.SE2Transform.identity())

dirname = d + "extractedlane"
draw_object(po, outdir = dirname)

### Get the width of one of the lane segments

In [ ]:
width = lane_segments_to_follow[0].width

### Merge the extracted lane segments to have one unified lane

In [ ]:
import itertools as it

# Make a list of all the control points, while making sure that the points that overlap are only taken once
lane_segments_to_follow_control_points = list(
    it.chain(
        *[ls.control_points[:-1] if ls is not lane_segments_to_follow[-1]
          else ls.control_points for ls in lane_segments_to_follow]
    )
)

# Creating a unified lane segment
lane_segments_to_follow_unified = dw.LaneSegment(
    width=width, control_points=lane_segments_to_follow_control_points
)

# Draw the unified lane segment
po = dw.PlacedObject()
po.set_object("Unified Lane", lane_segments_to_follow_unified, ground_truth=dw.SE2Transform.identity())
dirname = d + "unified_lane"
draw_object(po, outdir=dirname)

### Check if the merged lane has the same total length as the sum of its parts

In [ ]:
sum_lane_lengths = sum([ln.get_lane_length() for ln in lane_segments_to_follow])
sum_lane_segmenst_unified = lane_segments_to_follow_unified.get_lane_length()
msg = f"Lanes have not the same lenght: {sum_lane_lengths} is not {sum_lane_segmenst_unified}"
assert isclose(sum_lane_segmenst_unified , sum_lane_lengths, abs_tol=1e-5), msg

### All of the above has been unified in one function

In [ ]:
lane_segments_to_follow_unified = get_lane_from_node_sequence(m=duckie_map, node_sequence=[start, end])

# Draw the unified lane segment again
po = dw.PlacedObject()
po.set_object("Unified Lane", lane_segments_to_follow_unified, ground_truth=dw.SE2Transform.identity())
dirname = d + "unified_lane"
draw_object(po, outdir=dirname)

### It is also possible to give a longer node sequence (if nodes are skipped shortest path is computed)

In [ ]:
node_sequence = ['P30', 'P21', 'P10', 'P9']
lane_segments_to_follow_unified = get_lane_from_node_sequence(m=duckie_map, node_sequence=node_sequence)

# Draw the unified lane segment again
po = dw.PlacedObject()
po.set_object("Unified Lane", lane_segments_to_follow_unified, ground_truth=dw.SE2Transform.identity())
dirname = d + "unified_lane"
draw_object(po, outdir=dirname)

### Get a person to follow the lane

In [ ]:
big_duckie = Person(width / 2)

### Interpolate along the center line

In [ ]:
npoints = len(lane_segments_to_follow_unified.control_points) # get the control points of the lane
points = 15
betas = list(np.linspace(0, npoints-1, points))  # first control point beta=0, last control point beta=nbpoints-1

transforms = []
for beta in betas:
    p = lane_segments_to_follow_unified.center_point(beta)  # get interpolated pose in SE(2)
    transform = dw.SE2Transform.from_SE2(p)  # Transform to SE2Transform
    transforms.append(transform)

ground_truth = dw.SampledSequence[dw.SE2Transform](betas, transforms) # Setting the sequence for animation
duckie_map_lanefollow = deepcopy(duckie_map)  # copy the map  to prevent adding objects to the "original" map
duckie_map_lanefollow.set_object("Duckie", big_duckie, ground_truth=ground_truth) # add a duckie with the sequence
dirname = d + "lane_follow_unified"
draw_object(duckie_map_lanefollow, outdir=dirname)

### Define functions to interpolate along the centerlines when the position along the lane is known (in utils.py)

In [ ]:
from typing import List

def interpolate_along_lane(lane: dw.LaneSegment, along_lane: float) -> dw.SE2Transform:
    """ Input: lane and 1D position along the lane. Output: Pose on the duckietown map """
    dw_beta = lane.beta_from_along_lane(along_lane=along_lane)  # get the beta in in the dw representation
    p = lane.center_point(dw_beta)  # get pose
    transform = dw.SE2Transform.from_SE2(p)
    return transform

def interpolate_along_lane_n_points(
    lane: dw.LaneSegment,
    positions_along_lane: List[float]
) -> List[dw.SE2Transform]:
    """ Input: lane and sequence of 1D positions along the lane. Output: Pose sequence on the duckietown map """
    msg = f"Positions={positions_along_lane} have to be in ascending order to follow a lane"
    assert all(map(isclose, sorted(positions_along_lane), positions_along_lane)), msg
    transforms = [interpolate_along_lane(lane, along_lane) for along_lane in positions_along_lane]
    return transforms

### Test functions

In [ ]:
max_length = lane_segments_to_follow_unified.get_lane_length()  # get total length of the lane
points = 100  # number of points to interpolate
positions_along_lane = list(np.linspace(0, max_length, points))  # sequence of 1D position along the lane
timestamps = range(points) # just number of points
transforms = interpolate_along_lane_n_points(lane_segments_to_follow_unified, positions_along_lane)

# Animation
ground_truth = dw.SampledSequence[dw.SE2Transform](timestamps, transforms)
duckie_map_lanefollow = deepcopy(duckie_map)
duckie_map_lanefollow.set_object("Duckie", big_duckie, ground_truth=ground_truth)
dirname = d + "lane_follow_along_lane_interpolated"
draw_object(duckie_map_lanefollow, outdir=dirname)

### Define functions to interpolate along center line beta=0 start beta=1 end (in utils.py)

In [ ]:
def interpolate(lane: dw.LaneSegment, beta: float) -> dw.SE2Transform:
    """ Interpolate along the centerline of a lane. Start: beta=0, End beta=1 """
    lane_length = lane.get_lane_length() # get the length of the lane
    along_lane = beta * lane_length # get the corresponding position along the lane
    transform = interpolate_along_lane(lane=lane, along_lane=along_lane)
    return transform

def interpolate_n_points(lane: dw.LaneSegment, betas: List[float]) -> List[dw.SE2Transform]:
    """ Get pose sequence as a SE2Transform along the center line of a lane, beta=0 start beta=1 end """
    msg = f"betas = {betas} have to be in ascending order to follow a lane"
    assert all(map(isclose, sorted(betas), betas)), msg  # check if values are ascending
    transforms = [interpolate(lane, beta) for beta in betas]
    return transforms


### Test the functions

In [ ]:
points = 60 # number of interpolation points
betas = list(np.linspace(0,1,points))
timestamps = range(points)
transforms = interpolate_n_points(lane_segments_to_follow_unified, betas)  # get the pose sequence

# Animation
ground_truth = dw.SampledSequence[dw.SE2Transform](timestamps, transforms)
duckie_map_lanefollow = deepcopy(duckie_map)
duckie_map_lanefollow.set_object("Duckie", big_duckie, ground_truth=ground_truth)
dirname = d + "lane_follow_interpolated"
draw_object(duckie_map_lanefollow, outdir=dirname)


## Collection of other useful functions

### Sample some center line points of a lane (default 5 points)

In [ ]:
lane_points = lane_segments_to_follow_unified.center_line_points()  # in SE(2)
print(lane_points)

### Get the coordinates of the tile for a certain pose on the map

In [ ]:
q = lane_points[2]  # some pose
tile_at_point = get_tile_at_point(duckie_map, q)
print(tile_at_point)

### Get a collection of useful information for the pose in the map

In [ ]:
lane_poses = list(get_lane_poses(duckie_map, q))
print(lane_poses)

### Get the evaluation of a simulated sequence

In [ ]:
timestamps_as_floats = list(map(float, timestamps))  # timestamps have to be floats for it to work
evaluated_rules = evaluate_rules(
    poses_sequence=ground_truth,
    interval=dw.SampledSequence[dw.Timestamp](timestamps_as_floats, timestamps_as_floats),
    world=duckie_map,
    ego_name="Duckie"
)
for key, item in evaluated_rules.items():
    print(f"{key} : {item}\n")

### Get a LanePose for a certain pose with very detailed information

Information about the "position in the lane"
Am I inside the lane? If not why not?

In [ ]:
lane_pose = lane_segments_to_follow_unified.lane_pose_from_SE2(q)
print(lane_pose)
print("\n")
print(lane_pose.inside)
print("\n")
print(lane_pose.correct_direction)